# Predict real estate prices with Keras and TensorFlow

In [2]:
!pip install keras

  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl


In [3]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [5]:
dataframe = pd.read_csv("housing.csv", sep=',', header=0)
dataset = dataframe.values

In [6]:
dataframe.head()

,BIZPROP,ROOMS,AGE,HIGHWAYS,TAX,PTRATIO,LSTAT,VALUE
0,19.58,7.489,90.8,5,403,14.7,1.73,50.0
1,19.58,7.802,98.2,5,403,14.7,1.92,50.0
2,19.58,8.375,93.9,5,403,14.7,3.32,50.0
3,19.58,7.929,96.2,5,403,14.7,3.70,50.0
4,2.46,7.831,53.6,3,193,17.8,4.45,50.0


In [7]:
features = dataset[:,0:7]
target = dataset[:,7]

In [26]:
dataset.shape

(506, 8)

In [8]:
# fix random seed for reproducibility 
seed = 9 
numpy.random.seed(seed)

In [13]:
def simple_shallow_seq_net():
   # create a sequential ANN 
    model = Sequential()
    # linear stack of layers
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='sigmoid')) 
    # another layer with a single neuron initialized using a random normal distribution
    model.add(Dense(1, kernel_initializer='normal')) 
    sgd = optimizers.SGD(lr=0.01) 
    # mean squared error (MSE) cost function to measure the magnitude of the error rate of the model
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

The next step is to set a random seed for reproducibility; this random function is used to split the data into training and validation. The method used is **k-fold validation**, where the data is randomly divided into 10 subsets for training and validation:


In [14]:
seed = 9 
kfold = KFold(n_splits=10, random_state=seed)

Fit this model to predict a numerical value (house price, in this case), therefore we use KerasRegressor. KerasRegressor is a Keras wrapper used to access the regression estimators for the model from sklearn

In [16]:
estimator = KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100, batch_size=50, verbose=0)

In [27]:
# train and cross-validate across the subsets of the data and print the MSE
results = cross_val_score(estimator, features, target, cv=kfold) 
print("simple_shallow_seq_model:(%.2f) MSE" % (results.std()))

simple_shallow_seq_model:(162.76) MSE


In [28]:
estimator.fit(features, target)
estimator.model.save('simple_shallow_seq_net.h5')

Now, we try to improve its performance (lower the MSE) when we standardize the data and use it.
Create a pipeline to standardize the data and then use it during every learning cycle of the network. 

In [29]:
estimators = [] 
estimators.append(('standardize', StandardScaler())) 
estimators.append(('estimator', KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100, batch_size=50, verbose=0))) 
pipeline = Pipeline(estimators)

In [30]:
results = cross_val_score(pipeline, features, target, cv=kfold) 
print("simple_std_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_std_shallow_seq_net:(65.15) MSE


In [31]:
pipeline.fit(features, target) 
pipeline.named_steps['estimator'].model.save('standardised_shallow_seq_net.h5')

In [32]:
def deep_seq_net(): 
    # create a deep sequential model 
    model = Sequential() 
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='sigmoid')) 
    model.add(Dense(7,activation='tanh')) 
    model.add(Dense(7,activation='sigmoid')) 
    model.add(Dense(7,activation='tanh')) 
    model.add(Dense(1, kernel_initializer='normal')) 
    sgd = optimizers.SGD(lr=0.01) 
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [36]:
# Create the pipeline and fit the model using standardized data
estimators = [] 
estimators.append(('standardize', StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=deep_seq_net, epochs=100, batch_size=50, verbose=0))) 
pipeline = Pipeline(estimators)

In [37]:
results = cross_val_score(pipeline, features, target, cv=kfold) 
print("simple_std_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_std_shallow_seq_net:(57.74) MSE


In [38]:
pipeline.fit(features, target) 
pipeline.named_steps['estimator'].model.save('deep_seq_net.h5')

Now, let's see what happens when we widen the network, that is, increase the number of neurons (nodes) in each layer. 

In [39]:
def deep_and_wide_net(): 
    # create a deep sequential model 
    model = Sequential() 
    model.add(Dense(21, input_dim=7, kernel_initializer='normal', activation='sigmoid')) 
    model.add(Dense(21,activation='relu')) 
    model.add(Dense(21,activation='relu')) 
    model.add(Dense(21,activation='sigmoid')) 
    model.add(Dense(1, kernel_initializer='normal')) 
    sgd = optimizers.SGD(lr=0.01) 
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [40]:
estimators = [] 
estimators.append(('standardize', StandardScaler())) 
estimators.append(('estimator', KerasRegressor(build_fn=deep_and_wide_net, epochs=100, batch_size=50, verbose=0))) 
pipeline = Pipeline(estimators)

In [41]:
results = cross_val_score(pipeline, features, target, cv=kfold) 
print("deep_and_wide_model:(%.2f) MSE" % (results.std()))

deep_and_wide_model:(59.07) MSE


In [42]:
pipeline.fit(features, target) 
pipeline.named_steps['estimator'].model.save('deep_and_wide_net.h5')

# Serving the model as an API

In [43]:
!pip install Flask


In [44]:
#create a python file named  simple_api.py and type:
from flask import Flask, request 
app = Flask(__name__)

In [45]:
@app.route('/') 
def hello_world(): 
    return 'This is the Index page'

@app.route('/add', methods=['POST']) 
def add(): 
    req_data = request.get_json() 
    number_1 = req_data['number_1'] 
    number_2 = req_data['number_2'] 
    return str(int(number_1)+int(number_2))

Open terminal and type:

Windows

set FLASK_APP=simple_api

Linux and Mac

export FLASK_APP=simple_api


flask run